In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset

# Load the preprocessed dataset
df_reviews = pd.read_csv("../Data/processed/df_reviews.csv")

In [2]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(df_reviews['text'], df_reviews['sentiment'], test_size=0.2, random_state=42)

In [3]:
# Tokenization for RoBERTa
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings_roberta = tokenizer_roberta(X_train.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings_roberta = tokenizer_roberta(X_test.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")

In [4]:
# Tokenization for XLNet
tokenizer_xlnet = XLNetTokenizer.from_pretrained('xlnet-base-cased')
train_encodings_xlnet = tokenizer_xlnet(X_train.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings_xlnet = tokenizer_xlnet(X_test.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")

In [5]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [6]:
# Convert labels to numerical values
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
y_train_numeric = y_train.map(label_map).tolist()
y_test_numeric = y_test.map(label_map).tolist()

In [7]:
# Create datasets
train_dataset_roberta = CustomDataset(train_encodings_roberta, y_train_numeric)
test_dataset_roberta = CustomDataset(test_encodings_roberta, y_test_numeric)

train_dataset_xlnet = CustomDataset(train_encodings_xlnet, y_train_numeric)
test_dataset_xlnet = CustomDataset(test_encodings_xlnet, y_test_numeric)

In [8]:
# Define the RoBERTa model
model_roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Define the XLNet model
model_xlnet = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=3)


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [11]:
# Trainer for RoBERTa
trainer_roberta = Trainer(
    model=model_roberta,
    args=training_args,
    train_dataset=train_dataset_roberta,
    eval_dataset=test_dataset_roberta,
)

In [12]:
# Trainer for XLNet
trainer_xlnet = Trainer(
    model=model_xlnet,
    args=training_args,
    train_dataset=train_dataset_xlnet,
    eval_dataset=test_dataset_xlnet,
)

In [ ]:
# Train the models
trainer_roberta.train()

  0%|          | 0/7500 [00:00<?, ?it/s]

{'loss': 1.2277, 'grad_norm': 5.543136119842529, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 1.2015, 'grad_norm': 4.652154445648193, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}


In [ ]:
trainer_xlnet.train()